# ID3 (Decision Tree) from scratch

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## ID3

### Data

In [2]:
data = pd.read_csv('baseball.csv')
data

,Outlook,Temperature,Humidity,Wind,Play Golf
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [4]:
data.columns = ['outlook', 'temperature', 'humidity', 'wind', 'play']
data

,outlook,temperature,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


### Model

#### Entropy

In [5]:
def entropy(labels):
  p = labels.value_counts() / len(labels)
  ent = -sum(p * np.log2(p))
  return ent

In [6]:
entropy(data['play'])

0.9402859586706311

#### Information Gain

In [10]:
def information_gain(data, feature, target):
  entropy_parent = entropy(data[target])

  entropy_child = 0
  for value in data[feature].unique():
    subset = data[data[feature] == value]
    w = len(subset) / len(data)
    entropy_child += w * entropy(subset[target])

  return entropy_parent - entropy_child

In [11]:
information_gain(data, 'wind', 'play')

0.04812703040826949

In [12]:
information_gain(data, 'outlook', 'play')

0.24674981977443933

#### Decision Tree & Node

In [15]:
class Node:

  def __init__(self, feature=None, label=None):
    self.feature = feature
    self.label = label
    self.children = {}

  def __repr__(self):
    if self.feature is not None:
      return f'DecisionNode(feature="{self.feature}", children="{self.children}")'
    else:
      return f'LeafNode(label="{self.label}")'

In [28]:
def make_tree(data, target):
  # leaf node?
  if len(data[target].unique()) == 1:
    return Node(label=data[target].iloc[0])

  # calculate information gain
  features = data.drop(target, axis=1).columns
  gains = [information_gain(data, feature, target) for feature in features]

  # greedy search to find best feature
  max_gain_index = np.argmax(gains)
  best_feature = features[max_gain_index]

  # make node
  node = Node(feature=best_feature)

  # loop over the best feature
  for value in data[best_feature].unique():
    subset = data[data[best_feature] == value].drop(best_feature, axis=1)
    node.children[value] = make_tree(subset, target)

  return node

In [29]:
tree = make_tree(data, 'play')
tree

DecisionNode(feature="outlook", children="{'Sunny': DecisionNode(feature="humidity", children="{'High': LeafNode(label="No"), 'Normal': LeafNode(label="Yes")}"), 'Overcast': LeafNode(label="Yes"), 'Rain': DecisionNode(feature="wind", children="{'Weak': LeafNode(label="Yes"), 'Strong': LeafNode(label="No")}")}")

### Evaluation

In [31]:
data_test = pd.read_csv('baseball-test.csv')
data_test.columns = ['outlook', 'temperature', 'humidity', 'wind', 'play']
data_test

,outlook,temperature,humidity,wind,play
0,Overcast,Mild,High,Weak,Yes
1,Rain,Cool,Normal,Strong,No
2,Sunny,Hot,Normal,Weak,Yes


In [36]:
def predict(node, sample):
  if node.feature is None:
    return node.label

  feature_value = sample[node.feature]

  if feature_value in node.children:
    return predict(node.children[feature_value], sample)
  else:
    return node.label

In [37]:
[predict(tree, sample) for _, sample in data_test.iterrows()]

['Yes', 'No', 'Yes']